# Toy topology and signals dataset

In [345]:
import numpy as np 
from itertools import combinations
from tqdm import tqdm
from matplotlib import pyplot as plt

In [346]:
# Fourier basis + Canonical basis

def FB(D:int,
       over = False): 

    k = np.arange(D).reshape((D, 1))
    n = np.arange(D).reshape((1, D))

    if over:
        return np.hstack([(1/np.sqrt(D)) * np.exp(-2j * np.pi * k * n / D), np.eye(D)])
    else:
        return (1/np.sqrt(D)) * np.exp(-2j * np.pi * k * n / D)

In [347]:
def subSpaceAssignment(N:int,
                       D:int,
                       K:int):
    
    combo = list(combinations(range(D), K))
    patterns = np.random.choice(len(combo), N, replace=False)
    return {n: np.array(combo[patterns[n]]) for n in range(N)}

In [348]:
def premultiplier(Xu, Xv):
    uu = np.linalg.inv(Xu @ Xu.T)
    uv = Xu @ Xv.T
    vv = np.linalg.inv(Xv @ Xv.T)
    vu = Xv @ Xu.T

    return (uu, uv, vv, vu)

def chi_u(uu, uv, vv, vu):

    return ((uu @ uv - np.eye(uu.shape[0])) @ vv @ np.linalg.inv(vu @ uu @ uv @ vv - np.eye(uu.shape[0])) @ vu - np.eye(uu.shape[0])) @ uu

def chi_v(uu, uv, vv, vu):

    return (uu @ uv - np.eye(uu.shape[0])) @ vv @ np.linalg.inv(vu @ uu @ uv @ vv - np.eye(uu.shape[0]))

________________

In [349]:
V = 4
edges = [
    (0,1),
    (1,2),
    (1,3),
    (0,2)
]

nodes = [i for i in range(V)]
E = len(edges)

In [350]:
d = 20
k = 5
N = 100

In [351]:
D = FB(d)

In [352]:
subspaces = {0:np.array([0,1,2,3,4]),
             1:np.array([3,4,5,6,7]),
             2:np.array([1,2,3,4,5]),
             3:np.array([6,7,8,9,10])}

In [353]:
signals = {node: D[:,subspaces[node]] @ np.random.randn(k, N)
           for node in nodes}

In [354]:
T = 0

H = {
    edge : {
        edge[0] : None,
        edge[1] : None
    }
for edge in combinations(nodes, 2)
}

In [355]:
for e in tqdm(combinations(nodes,2)):
    u = e[0]
    v = e[1]

    X_u = signals[u]
    X_v = signals[v]
    uu, uv, vv, vu = premultiplier(X_u, X_v)

    H[e][u] = chi_u(uu, uv, vv, vu)
    H[e][v] = chi_u(uu, uv, vv, vu)
    
    T += np.trace(H[e][u]) + np.trace(H[e][v])

6it [00:00, 120.00it/s]


In [356]:
mu = T

In [357]:
H = {
    edge : {
        edge[0] : mu/T * (H[edge][edge[0]]),
        edge[1] : mu/T * (H[edge][edge[1]])
    }
for edge in combinations(nodes, 2)
}

In [358]:
all_edges = list(combinations(range(V), 2))

energies = {
    e : 0
    for e in all_edges
    }

for e in (all_edges):
    u = e[0]
    v = e[1]
    
    '''
    X_ = np.zeros_like(X)
    X_[u*d:(u+1)*d,:] = X[u*d:(u+1)*d,:]
    X_[v*d:(v+1)*d,:] = X[v*d:(v+1)*d,:]
    energies[e] = np.linalg.norm(BB @ X_)
    '''

    energies[e] = np.linalg.norm(H[e][e[0]] @ signals[e[0]] - H[e][e[1]] @ signals[e[1]])

In [359]:
retrieved = sorted(energies.items(), key=lambda x:x[1])[:E]

In [360]:
print(f'Accuracy in retrieving underlying graph {len(set(list(map(lambda x: x[0], retrieved))).intersection(set(edges))) / E}')

Accuracy in retrieving underlying graph 1.0


In [361]:
retrieved

[((0, 2), 3345220096629341.0),
 ((1, 3), 3740276750234392.5),
 ((0, 1), 4435863540478665.5),
 ((1, 2), 6218057716780081.0)]